In [ ]:
import codecs
import os
import os.path
import shutil
import string
import sys
import warnings
from typing import Any, Callable, Dict, List, Optional, Tuple
from urllib.error import URLError
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
import numpy as np
import torch
from PIL import Image
import json
from torchvision.io import read_image
from torchvision.transforms import ToTensor, Normalize
import torchvision.transforms as transforms


#from .utils import download_and_extract_archive, extract_archive, verify_str_arg, check_integrity
from torchvision.datasets.vision import VisionDataset
import pandas as pd
import glob

class BIWI(VisionDataset):


    training_file = "training.pt"
    test_file = "test.pt"


    @property
    def train_labels(self):
        warnings.warn("train_labels has been renamed targets")
        return self.targets

    @property
    def test_labels(self):
        warnings.warn("test_labels has been renamed targets")
        return self.targets

    @property
    def train_data(self):
        warnings.warn("train_data has been renamed data")
        return self.data

    @property
    def test_data(self):
        warnings.warn("test_data has been renamed data")
        return self.data

    def __init__(
        self,
        root: str = None,
        train: bool = True,
        transform: Optional[Callable] = None,
        target_transform: Optional[Callable] = None,
        split : float = 1
      
    ) -> None:
        super().__init__(root, transform=transform, target_transform=target_transform)
        self.train = train  # training set or test set
        self.split = split
        self.height=96
        self.width=96

        

        
	
        self.json_files = self._load_data()
 
        

        
    def _load_data(self,mypath = "/home/nandhini/Animals"):
        onlyfiles_json = []
        
        for data in glob.glob(mypath+"/*.json"):
            onlyfiles_json.append(data)
            
        # calculate the validation data sample length
        valid_split = int(len(onlyfiles_json) * self.split)
        # calculate the training data samples length
        train_split = int(len(onlyfiles_json) - valid_split)  
        
        if self.train == True:
            onlyfiles_json = onlyfiles_json[:train_split]
            print(f"Training sample instances: {len(onlyfiles_json)}")
        elif self.train == False:
            onlyfiles_json = onlyfiles_json[-valid_split:]
            print(f"Validation sample instances: {len(onlyfiles_json)}")
        
        return onlyfiles_json
       
    def __getitem__(self, index: int) -> Tuple[Any, Any]:
        """
        Args:
            index (int): Index

        Returns:
            tuple: (image, target) where target is index of the target class.
        """
        
        data = json.load(open(self.json_files[index]))
        img_path = self.json_files[index].split('.')[0] + '.jpeg'
        target = data["shapes"][0]["points"][0]
        


        img = Image.open(img_path)
        res_x, res_y = img.size
        img = img.resize((self.height,self.width), Image.ANTIALIAS)
        convert = ToTensor()
        img = convert(img)


        target = [target[0]/res_x, target[1]/res_y]


        target = torch.Tensor(target)
        

        if self.transform is not None:
            img = self.transform(img)

        if self.target_transform is not None:
            target = self.target_transform(target)
            

       
        return img, target



    def __len__(self) -> int:
        return len(self.json_files)
    
    @property
    def raw_folder(self) -> str:
        return os.path.join(self.root, self.__class__.__name__, "raw")

    @property
    def processed_folder(self) -> str:
        return os.path.join(self.root, self.__class__.__name__, "processed")

    @property
    def class_to_idx(self) -> Dict[str, int]:
        return {_class: i for i, _class in enumerate(self.classes)}

    def _check_exists(self) -> bool:
        return all(
            check_integrity(os.path.join(self.raw_folder, os.path.splitext(os.path.basename(url))[0]))
            for url, _ in self.resources
        )

    def download(self) -> None:
        """Download the MNIST data if it doesn't exist already."""

        if self._check_exists():
            return

        os.makedirs(self.raw_folder, exist_ok=True)

        # download files
        for filename, md5 in self.resources:
            for mirror in self.mirrors:
                url = f"{mirror}{filename}"
                try:
                    print(f"Downloading {url}")
                    download_and_extract_archive(url, download_root=self.raw_folder, filename=filename, md5=md5)
                except URLError as error:
                    print(f"Failed to download (trying next):\n{error}")
                    continue
                finally:
                    print()
                break
            else:
                raise RuntimeError(f"Error downloading {filename}")

    def extra_repr(self) -> str:
        split = "Train" if self.train is True else "Test"
        return f"Split: {split}"

    # get the training and validation data samples
print('\n-------------- PREPARING DATA --------------\n')
train_data=BIWI(train = True)
valid_data = BIWI(train = False)

print('\n-------------- DATA PREPRATION DONE --------------\n')
   
    # prepare data loaders
train_loader = DataLoader(train_data,
                              batch_size=256,
                              shuffle=False)
valid_loader = DataLoader(valid_data,
                              batch_size=256,
                              shuffle=False)